In [ ]:
import socket, pandas
from pyspark.sql import SparkSession
import pyspark

In [ ]:
pod_template = "./spark-pod-template.yaml"

In [ ]:
sparkSession = SparkSession.builder \
.config('spark.master', 'k8s://https://kubernetes.default.svc.cluster.local') \
.config("spark.executor.instances","8") \
.config("spark.executor.memory","4G") \
.config("spark.executor.cores","4") \
.config("spark.kubernetes.executor.request.cores","250m") \
.config("spark.kerberos.keytab","/etc/secrets/keytab") \
.config("spark.kerberos.principal",open('/etc/secrets/principal', 'r').read()) \
.config("spark.kubernetes.namespace",open('/var/run/secrets/kubernetes.io/serviceaccount/namespace','r').read()) \
.config("spark.kubernetes.driver.annotation.sidecar.istio.io/inject", "false") \ ## Used if istio is deployed for example in Kubeflow (remove comments to run without error)
.config("spark.kubernetes.executor.annotation.sidecar.istio.io/inject", "false") \ ## Used if istio is deployed for example in Kubeflow
.config("spark.kubernetes.driver.podTemplateFile", pod_template) \ ## Use this pod_template to add tolerations + node_selector
.config("spark.kubernetes.executor.podTemplateFile", pod_template) \ ## Use this pod_template to add tolerations + node_selector
.config("spark.kubernetes.container.image", )
.config("spark.driver.host", socket.gethostbyname(socket.gethostname())) \
.appName("Spark On Kube") \
.enableHiveSupport() \
.getOrCreate()